<a href="https://colab.research.google.com/github/USTCJiuhu/esp32/blob/master/Materials_Project_Lab_DFT_in_practice_using_pymatgen_and_VASP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial – DFT in practice using pymatgen and VASP
## Follow along at https://bit.ly/2WesJmp (examples filled in) or https://colab.research.google.com/drive/1tQQd0mfC_vXB1ftWXZspJoOIIgWpXTGI (fill in the blanks).
## Use "File > Save a copy in Drive..." to create your own copy to edit.

# Introduction to VASP

VASP (https://www.vasp.at) is:

* An industry-leading DFT code for periodic systems (this means a plane-wave basis)
* Developed at University of Vienna (Vienna Ab-initio Simulation Package), proprietary code ($$$)
* Fast, (generally) reliable, trusted
* Created ~1989, latest release in 2018, actively developed

Alternatives to VASP: Quantum Espresso,  ABINIT, CASTEP, many more…

# Essential ingredients for a VASP calculation

You need **four files**, these are:

1. A **POSCAR** file, defining the positions of the atoms and the lattice parameters of your cell
2. An **INCAR** file, defining calculation settings
3. A **KPOINTS** file, defining your integration points in the Brillouin zone
4. A **POTCAR** file, defining your pseudopotentials

# What is pymatgen, and why is it helpful?

Pymatgen (https://pymatgen.org) can take in your crystal structure and help create your **INCAR**, **POSCAR**, **KPOINTS** and **POTCAR** for you, while avoiding many common pitfalls and errors.

Pymatgen also supports many **common crystallographic transformations** to modify your crystal structure for you. This will be important in a later lab for the bulk modulus calculation which requires doing calculations on a series of deformed crystal structures corresponding to different strains.

Finally, pymatgen can help **analyze the output** of a VASP calculation to make it easier to process, extract useful information from, or incorporate into graphs or reports.

# Is this useful to know if I'm not intending to run any DFT personally?

*Maybe*, knowledge of using pymatgen can help:

* Access the Material Project's database of over 130,000 materials and their associated properties
* Perform many analysis tasks such as generating phase diagrams, Pourbaix (aqueous stability), band structures, etc.
* **Can provide insight into the correct settings for a DFT calculation** if you're reviewing a paper

# 1. Install and import pymatgen

First, let's verify we have pymatgen installed. The following command should produce no error or warning:

In [0]:
import pymatgen

In a Jupyter notebook such as Google Colaboratory, installing pymatgen is often as easy as running `!pip install pymatgen` (note the leading !), so let's do that now.

In [0]:
!pip install pymatgen

Now if we import pymatgen it should work correctly:

In [0]:
import pymatgen

And we can also check the version of pymatgen we hvae installed:

In [0]:
print(pymatgen.__version__)

For a list of new features, bug fixes and other changes, consult the [changelog on pymatgen.org](http://pymatgen.org/change_log.html). Note that following the example of the larger Python community, pymatgen only supports Python 3+ as of 2019.

# 2. Download some example data

For this tutorial, we've prepared some example data to play with. Let's download it into our notebook by evaluating this next cell.

In [0]:
def download_example_files():
    
    from urllib.request import urlretrieve
    from os.path import basename
    
    example_files = ('LiFePO4.cif',
                     'fe_monomer/OUTCAR', 'fe_monomer/vasprun.xml',
                     'vasprun.xml.unconverged', 'vasprun_Si_bands.xml')
    
    root_path = "https://raw.githubusercontent.com/materialsproject/pymatgen/master/test_files"
    
    for file in example_files:
        urlretrieve(f"{root_path}/{file}", basename(file))
        print(f"Downloaded {file}")
    
    print("All example files downloaded successfully!")

download_example_files()

# 3. Introducing Structures and Molecules, the fundamental building blocks in pymatgen

Most of the fundamentals of pymatgen are expressed in terms of [**`Structure`**](http://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.Structure) and [**`Molecule`**](http://pymatgen.org/pymatgen.core.structure.html#pymatgen.core.structure.Molecule) objects.

While we will mostly be using `Structure`, `Stucture` and `Molecule` are very similar conceptually. The main difference is that `Structure` supports full periodicity required to describe crystallographic structures.

## 3.1 Creating a `Structure` and `Lattice`

In [0]:
from pymatgen import Structure, Lattice

A `Lattice` can be created in one of several ways. Such as by inputting a 3x3 matrix describing the individual lattice vectors. For example, a cubic lattice of length 5 Ångstrom:



In [0]:
my_lattice = Lattice([[5, 0, 0], [0, 5, 0], [0, 0, 5]])

In [0]:
my_lattice

Equivalently, we can create it from its lattice parameters:

In [0]:
my_lattice_2 = Lattice.from_parameters(5, 5, 5, 90, 90, 90)  # a, b, c, alpha, beta, gamma

Or, since we know in this case we have a cubic lattice, so a == b == c and alpha == beta == gamma == 90 degrees, we can simply put:

In [0]:
my_lattice_3 = Lattice.cubic(5)

In all cases, these lattices are the same:

In [0]:
my_lattice == my_lattice_2 == my_lattice_3

Now, we can create a crystal structure very easily. Let's start with body-centered-cubic iron, which in its conventional setting is a cubic lattice with a two atom basis:

In [0]:
fe = Structure(Lattice.cubic(2.8), ["Fe", "Fe"], [[0, 0, 0], [0.5, 0.5, 0.5]])

In [0]:
print(fe)

We provide three arguments to create this Structure object: the first is a Lattice, the second is a list of species (here, two irons), and then a list of the positions of the sites containing these species in *fractional co-ordinates*.

It's also possible to create an equivalent `Structure` using Cartesian co-ordinates:

In [0]:
fe_from_cart = Structure(Lattice.cubic(2.8), ["Fe", "Fe"], [[0, 0, 0], [1.4, 1.4, 1.4]],
                             coords_are_cartesian=True)

In [0]:
print(fe_from_cart)

We see check that both structures are equivalent:

In [0]:
fe == fe_from_cart

We can then access many properties of the structure, such as its volume:

In [0]:
print("Volume in Ångstroms^3", fe.volume)
print("Space group", fe.get_space_group_info())

Or to access the first site (note that Python is a 0-indexed programming language, so the first site is site 0):

In [0]:
fe[0]

## 3.2 Modifying a Structure

We can create a supercell by multiplying the structure by a number of repeats:

In [0]:
fe_repeated = fe*(2,2,2)

In [0]:
fe_repeated

There are many methods to modify the structure, such as scaling the volume or substituting one species with another species. There are also various *transformations* in pymatgen that can do more complicated structure manipulations, such as creating surfaces, grain boundaries or creating ordered approximations of disordered structure.

In [0]:
fe_primitive = fe.get_primitive_structure()

In [0]:
fe_primitive

## 3.3 Creating Structure from Spacegroups

Structures can also be created directly from their spacegroup:

In [0]:
bcc_fe = Structure.from_spacegroup("Im-3m", Lattice.cubic(2.8), ["Fe"], [[0, 0, 0]])
print(bcc_fe)

In this next example, we annotate our species with oxidation state, that is we specifiy `Na+` instead of `Na` and `Cl-` instead of `Cl`. Pymatgen understands  the concept of oxidation state, and will convert these strings as appropriate.

In [0]:
nacl = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5.692), ["Na+", "Cl-"],
                                 [[0, 0, 0], [0.5, 0.5, 0.5]])
print(nacl)

## 3.4 Creating a Disordered Structure

**Disordered** structures, that is structures with a mix of two or more types of species on a single site, are not appropriate for DFT calculations. However, experimental structures are often provided as disordered structures, and pymatgen has the capability to understand these. Here, as an example, we create a CuAu solid solution:

In [0]:
composition = {"Cu": 0.5, "Au":0.5}
cu_au = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [composition], [[0, 0, 0]])
print(cu_au)

## 3.5 Input/output from other standard file formats

Pymatgen supports a wide range of input/output formats, if it's in common usage pymatgen probably supports it. We will demonstrate loading a structure from a CIF file:

In [0]:
struct = Structure.from_file('LiFePO4.cif')
print(struct)

It is also possible to load structures in pymatgen directly from the Materials Project via their Materials Project ID (e.g. mp-13 is bcc Fe, https://materialsproject.org/mp-13) and the Materials Proejct API but this is not covered in this tutorial. 

# 4. Introducing VASP Input Sets

Now we have a crystal structure created, let's examine how to create the **POSCAR**, **INCAR**, **KPOINTS** and **POTCAR** files VASP requires.

To do this, pymatgen has the concept of a "VASP Input Set", where multiple sets exist depending on the type of calculation you want to perform. Some of the main ones are:

* `MPRelaxSet` -- the standard input set, this will set up a calculation that allows atoms to relax to their equilibirum positions and lattice parameters to relax
* `MPStaticSet` -- this is just to calculate the ground state electron density, keeping all atoms fixed, typically useful prior to a bandstructure calculation or for more accurate energies than an `MPRelaxSet`
* `MPNonSCFSet` -- this is to generate a band structure or density of states, typically this is done at much higher k-point densities but without self-consistence being enforced
* `MPSOCSet` -- a more expensive input set for when spin-orbit coupling has to be considered
* `MPMDSet`  -- an input set for ab initio molecular dynamics

...and more.

These are often chained together, e.g. `MPRelaxSet` --> `MPStaticSet` --> `MPNonSCFSet`. Our other Python code, `atomate`, helps chain these calculations together and run them automatically.

## 4.1 Creating a VASP Input Set

In [0]:
from pymatgen.io.vasp.sets import MPRelaxSet

In [0]:
vis = MPRelaxSet(fe)

From here, all the VASP input files can be written using `vis.write_input(...)` but let's look at each input file in turn. To run the calculation, you would then simply run the command `vasp` (or an equivalent in multiprocessing environments, such as `mpirun vasp`) inside the directory containing these files.

## 4.2 Examining the POSCAR

The POSCAR contains the positions and types of all the atoms and the lattice parameters and can eb read as follows:

In [0]:
vis.poscar

## 4.4 Examining the INCAR

The INCAR file contains various calculation settings ('tags'). These tags will not mean much without reference to the [VASP manual](https://cms.mpi.univie.ac.at/wiki/index.php/Category:INCAR); there are lots of 'magic numbers.'

In [0]:
vis.incar

Most tags have default values, so only tags that we want to set explicitly are listed here.

* **ALGO:** Electronic minimization algorithm
* **EDIFF:** Criteria after which the electronic minimization is stopped, marking the end of the electronic loop
* **ENCUT:** Energy cut-off for the plane wave basis in eV
* **IBRION:** Algorithm for how to update the positions of ions once the electronic loop completes
* **ICHARG:** Controls how the initial charge density is set before electronic minimization starts
* **ISIF:**  Specifies degrees of freedom (are atoms fixed, lattice parameters fixed, both, neither, etc.)
* **ISMEAR:** Controls how integration is preformed between k-points
* **ISPIN:** Whether to do a spin-polarized (i.e. allowing for magnetic moments) or non-spin-polarized calculation
* **LORBIT:** Does not change calculation, but does change what final analysis is done and what files are generated
* **LREAL:** Advanced setting (see manual for more information)
* **LWAVE:** Does not change calculation, but determines whether or not to write the full wavefunction at the end of the calculation
* **MAGMOM:** Related to `ISPIN`
* **NELM:** Maximum number of electronic steps considered, typically only need 3-7, 99 is very high!
* **NSW:** Maximum number of ionic steps
* **PREC:** Influences a variety of other settings, such as the dimensions of the FFT grid
* **SIGMA:** Related to `ISMEAR`

There are 278 in total! The purpose of the input set is so that you don't, in general, have to think about these. However note that some are very important for the resulting quality of your calculation and also for its speed.

## 4.4 Examining the KPOINTS

The KPOINTS file specifies the integration points in your Brillouin zone. They can be given explicitly, or can be specified as a uniform grid of points. 

In [0]:
vis.kpoints

In [0]:
# to visualize an grid in reciprocal space
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import plot_brillouin_zone
sga = SpacegroupAnalyzer(fe)
kpts = [k[0] for k in sga.get_ir_reciprocal_mesh((8,8,8))]
print("Number of k-points in irreducible Brillouin zone:", len(kpts))
plot_brillouin_zone(bcc_fe.lattice.reciprocal_lattice, kpoints=kpts)

## 4.5 Examining the POTCAR (?!)

The POTCAR file contains information on the *pseudopotentials* used for your calculations. These speed up the calculation by allowing the nucleus and core electrons to be considered together as an effective potential, so that only the valence electrons involved in bonding are calculated.

Unfortunately, VASP POTCAR files are proprietary so cannot be shared here, which is why this produces an error.

In [0]:
#vis.potcar  # this won't work unless you have a directory of POTCAR files available

## 4.5 Try a different structure

Note that the VASP input set parameters are structure-specific. If we generate a new input with our larger supercell, we note that the k-point mesh is correspondingly reduced.

In [0]:
MPRelaxSet(fe_repeated).kpoints

## 4.6 Try a different input set

This input set is used for a band structure calculation.

In [0]:
from pymatgen.io.vasp.sets import MPNonSCFSet

In [0]:
vis = MPNonSCFSet(fe_primitive)

In [0]:
vis.kpoints

We can also visualize these kpoints, and see that they draw a line through reciprocal space.

In [0]:
from pymatgen.electronic_structure.plotter import plot_brillouin_zone
plot_brillouin_zone(vis.structure.lattice.reciprocal_lattice, labels={label:coords for label, coords in zip(vis.kpoints.labels, vis.kpoints.kpts)}, kpoints=vis.kpoints.kpts)

# 5 Examining VASP Output

There are two main outputs for any successfully completed VASP calculation: an OUTCAR file and a vasprun.xml. Both files contain essentially the same information, but the OUTCAR file is more human readable, 

In [0]:
from pymatgen.io.vasp.outputs import Vasprun, Outcar

In [0]:
vasprun = Vasprun('vasprun.xml')
outcar = Outcar('OUTCAR')

We can examine various outputs from the calculation by accessing properties of the Vasprun object:

In [0]:
vasprun.kpoints

In [0]:
vasprun.actual_kpoints

In [0]:
vasprun.final_structure

In [0]:
vasprun.final_energy

And gain information on how the calculation proceeded and how convergence was reached:

In [0]:
[v['e_0_energy'] for v in vasprun.ionic_steps[0]['electronic_steps']]

Not all information is in both the OUTCAR and vasprun.xml, for example run stats:

In [0]:
outcar.run_stats

## 5.1 Example of an unconverged run

Let's examine an unconverged Vasprun to see how it differs:

In [0]:
vasprun_unconverged = Vasprun('vasprun.xml.unconverged')

In [0]:
vasprun_unconverged.converged_ionic

In [0]:
vasprun_unconverged.converged_electronic

In [0]:
[v['e_0_energy'] for v in vasprun_unconverged.ionic_steps[0]['electronic_steps']]

# 5.2 Example of a band structure

Let's see an band structure

In [0]:
vasprun_bands = Vasprun('vasprun_Si_bands.xml')

In [0]:
vasprun_bands.final_structure

In [0]:
bs = vasprun_bands.get_band_structure()

In [0]:
bs.get_band_gap()

# Going Further

The following examples are a selection of the other capabilities of pymatgen. Please consult the [pymatgen documentation](https://pymatgen.org) or our [example notebooks](https://matgenb.materialsvirtuallab.org) if you're interested in learning more!

If you have some analysis that pymatgen does not support, but you feel would be a good fit, please talk to the developers. If you would like to contribute code to pymatgen yourself, you are also encouraged to do so and guidance is available.[link text](https://)

# Ex1. Symmetry Analysis with SymmetryAnalyzer

In addition to book-keeping of structures using `Structure` objects, pymatgen contains powerful tools for analyzing crystal symmetry and comparing structures.  The `SymmetryAnalyzer` object uses the powerful spglib symmetry analysis library, which is written in C for more efficient determination of invariant symmetry operations and crystal symmetry. 
The symmetry analyzer can be used to get primitive and standardized conventional cell settings of structures.

These examples shows how to get the primitive structure of BCC iron using `SpacegroupAnalyzer` and how to get the point group of the CO molecule created above using `PointGroupAnalyzer`.

In [0]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [0]:
sga = SpacegroupAnalyzer(fe)
prim = sga.get_primitive_standard_structure()
print(prim)  # note the primitive structure has only a single site

In [0]:
std = sga.get_conventional_standard_structure()  # whereas the conventional structure has two
print(std)

In [0]:
print("Crystal system:", sga.get_crystal_system())
print("Spacegroup symbol:", sga.get_space_group_symbol())

# Ex2. Example of calculating X-ray Diffraction (XRD) Pattern

There are various tools in pymatgen that allow for the analysis and plotting of structural and electronic information.  The `XRDCalculator` is perhaps the most straightforward of these tools, since it only requires a `Structure` object.

In [0]:
from pymatgen.analysis.diffraction.xrd import XRDCalculator
xrdc = XRDCalculator()

In [0]:
xrdc.get_plot(fe_primitive)  # plot the XRD pattern of NaCl

# Ex3. Example: Creating a surface

Here, we show how to generate all of the low-index facets for BCC Fe.

In [0]:
from pymatgen.core.surface import generate_all_slabs

In [0]:
slabs = generate_all_slabs(fe, 1, 4, 10)

In [0]:
first_slab = slabs[0]
print(first_slab)

In [0]:
for slab in slabs:
    print(slab.miller_index)